In [47]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [48]:
energy = pd.read_csv("D:\Semester-7\Data Visualisation\energy-dashboard\data\owid_energy.csv")
co2 = pd.read_csv("D:\Semester-7\Data Visualisation\energy-dashboard\data\owid_co2.csv")

In [49]:
cols = [
    "country",
    "year",
    "population",
    "primary_energy_consumption",
    "renewables_share_energy",
    "fossil_share_energy",
    "solar_share_energy",
    "wind_share_energy",
    "hydro_share_energy"
]

co2 = co2[[
    "country", "year",
    "co2"
]]

# Merge datasets
df = pd.merge(energy, co2, on=["country", "year"], how="left")

MAJOR_COUNTRIES = ['United States', 'Germany', 'China', 'France', 'United Kingdom', 'Japan', 'India', 'Canada']
df = df[df['country'].isin(MAJOR_COUNTRIES)]

# Filter years
df = df[df["year"] >= 1990]

# Remove aggregate regions
exclude = [
    "World", "Asia", "Europe", "Africa",
    "North America", "South America", "European Union"
]
df = df[~df["country"].isin(exclude)]

# Only keep countries that have at least some data
valid_countries = df.groupby('country')['primary_energy_consumption'].count()
valid_countries = valid_countries[valid_countries > 20].index.tolist()
df = df[df['country'].isin(valid_countries)]


In [51]:
app = dash.Dash(__name__)
app.title = "Global Energy Dashboard"

app.layout = html.Div(style={"padding":"20px"}, children=[
    html.H1("Global Energy Consumption & Renewable Energy Dashboard"),
    html.Hr(),

    html.Label("Select Country"),
    dcc.Dropdown(
        id='country',
        options=[{'label':c,'value':c} for c in sorted(df['country'].unique())],
        value='United States',
        clearable=False
    ),
    html.Br(),

    html.Label("Select Year Range"),
    dcc.RangeSlider(
        id='years',
        min=df['year'].min(),
        max=df['year'].max(),
        value=[2000, df['year'].max()],
        marks={y:str(y) for y in range(1990,2025,5)},
        step=1
    ),
    html.Br(),

    html.Label("Energy Type for Trend"),
    dcc.RadioItems(
        id='energy_type',
        options=[
            {'label':'Renewables','value':'renewables_share_energy'},
            {'label':'Fossil Fuels','value':'fossil_share_energy'},
            {'label':'Nuclear','value':'nuclear_share_energy'}
        ],
        value='renewables_share_energy',
        inline=True
    ),
    html.Br(), html.Br(),

    # Charts
    dcc.Graph(id='line_consumption'),
    dcc.Graph(id='line_energy_type'),
    dcc.Graph(id='line_co2'),
    dcc.Graph(id='bar_renewable'),
    dcc.Graph(id='map_renewable'),
    dcc.Graph(id='stacked_area'),
    dcc.Graph(id='scatter_co2_energy')
])

In [ ]:
@app.callback(
    Output('line_consumption','figure'),
    Output('line_energy_type','figure'),
    Output('line_co2','figure'),
    Output('bar_renewable','figure'),
    Output('map_renewable','figure'),
    Output('stacked_area','figure'),
    Output('scatter_co2_energy','figure'),
    Input('country','value'),
    Input('years','value'),
    Input('energy_type','value')
)
def update_charts(country, years, energy_type):
    # ---------------------
    # Country-specific data
    # ---------------------
    country_df = df[(df['country']==country) & (df['year'].between(years[0],years[1]))]

    # 1️⃣ Primary Energy Line
    fig1 = px.line(country_df.dropna(subset=['primary_energy_consumption']),
                   x='year', y='primary_energy_consumption',
                   title=f'Primary Energy Consumption ({country})',
                   labels={'primary_energy_consumption':'Energy Consumption (TWh)'})

    # 2️⃣ Selected Energy Type Line
    fig2 = px.line(country_df.dropna(subset=[energy_type]),
                   x='year', y=energy_type,
                   title=f'{energy_type.replace("_"," ").title()} Trend ({country})',
                   labels={energy_type:'Energy Share (%)'})

    # 3️⃣ CO2 Emissions Line
    fig3 = px.line(country_df.dropna(subset=['co2']),
                   x='year', y='co2',
                   title=f'CO₂ Emissions ({country})',
                   labels={'co2':'CO₂ Emissions (million tonnes)'})

    # ---------------------
    # Global charts (ignore country filter)
    # ---------------------
    global_df = df[df['year'].between(years[0],years[1])]
    latest_year = years[1]

    # 4️⃣ Top 15 Renewable Share Bar
    bar_df = global_df[global_df['year']==latest_year].dropna(subset=['renewables_share_energy'])
    bar_df = bar_df.sort_values('renewables_share_energy',ascending=False).head(15)
    fig4 = px.bar(bar_df,x='country',y='renewables_share_energy',
                  title=f'Top Countries by Renewable Share ({latest_year})',
                  labels={'renewables_share_energy':'Renewable Share (%)'})

    # 5️⃣ Renewable Share Map
    map_df = global_df[global_df['year']==latest_year].dropna(subset=['renewables_share_energy'])
    fig5 = px.choropleth(map_df, locations='country', locationmode='country names',
                         color='renewables_share_energy', color_continuous_scale='Viridis',
                         title=f'Global Renewable Share ({latest_year})',
                         hover_name='country',
                         labels={'renewables_share_energy':'Renewable Share (%)'})

    # 6️⃣ Stacked Area Chart
    stacked_df = country_df.dropna(subset=['renewables_share_energy','fossil_share_energy','nuclear_share_energy'])
    fig6 = px.area(stacked_df, x='year',
                   y=['renewables_share_energy','fossil_share_energy','nuclear_share_energy'],
                   title=f'Stacked Energy Share Over Time ({country})',
                   labels={'value':'Energy Share (%)','variable':'Energy Type'})

    # 7️⃣ Scatter CO2 vs Energy Consumption
    latest_year = years[1]
    scatter_df = df[df['year']==latest_year].dropna(subset=['co2','primary_energy_consumption','population'])
    fig7 = px.scatter(
        scatter_df,
        x='primary_energy_consumption',
        y='co2',
        size='population',
        color='country',
        hover_name='country',
        title=f'CO₂ vs Energy Consumption ({latest_year})',
        labels={
            'primary_energy_consumption':'Energy Consumption (TWh)',
            'co2':'CO₂ Emissions (million tonnes)'
        }
    )

    return fig1, fig2, fig3, fig4, fig5, fig6, fig7

# =============================
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
